This file, once complete, will obtain a dataframe that contains the cost per point table from CF for the 2023-24 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per Point of NHL Players for the 2023-24 season. So I am going to scrape the cap friendly website (Cost Per Point tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-point/2024"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(5)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_26196\3267800375.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
dfattempt.shape
dfattempt.tail(5)

,#,PLAYER,POS,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,SH,G,A,P,TOI,$/SH,$/G,$/A,$/P,$/MIN
816,817,"McGing, Hugh","LW, C",25,NaN,Stnd,UFA,"$775,000",5,4,0,0,0,08:22,"$193,750",∞,∞,∞,"$18,525"
817,818,"Bortuzzo, Robert",RD,34,NaN,Stnd,UFA,"$950,000",27,11,0,0,0,13:43,"$86,363",∞,∞,∞,"$2,565"
818,819,"Rosén, Isak",RW,20,NaN,ELC,RFA,"$894,167",7,8,0,0,0,08:35,"$111,770",∞,∞,∞,"$14,882"
819,820,"Lavoie, Raphael","RW, C",23,NaN,Stnd,RFA,"$874,125",7,4,0,0,0,07:17,"$218,531",∞,∞,∞,"$17,145"
820,821,"Peterson, Jacob","LW, C",24,NaN,Stnd,RFA,"$775,000",6,3,0,0,0,13:15,"$258,333",∞,∞,∞,"$9,748"


In [4]:
dfattempt.to_csv("CF Cost Per Point 2023-24.csv") 